# Regression with ARIMA Errors.

This model is a simple Regression model with ARIMA errors.  The regression model consists of a muttiple independent variables: seasonal indicies (month and day of week) and a special event: 'new years day'.  This is a categorical variable to model the special event on 1st Jan every year.

ARIMA order is chosen by `pmdarima.auto_arima`

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from forecast_tools.metrics import (mean_absolute_scaled_error, 
                                    root_mean_squared_error,
                                    symmetric_mean_absolute_percentage_error)

#auto_arima
from pmdarima import auto_arima

from statsmodels.tsa.arima.model import ARIMA

import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels as sm
sm.__version__

'0.11.0'

In [3]:
import pmdarima as pm
pm.__version__

'1.5.2'

In [4]:
from amb_forecast.feature_engineering import (featurize_time_series,
                                              regular_busy_calender_days)

# Data Input

In [5]:
TOP_LEVEL = '../../../results/model_selection'
STAGE = 'stage1'
REGION = 'Trust'
METHOD = 'reg-arima2'

FILE_NAME = 'Daily_Responses_5_Years_2019_full.csv'

#split training and test data.
TEST_SPLIT_DATE = '2019-01-01'

#second subdivide: train and val
VAL_SPLIT_DATE = '2017-07-01'

#discard data after 2020 due to coronavirus
#this is the subject of a seperate study.
DISCARD_DATE = '2020-01-01'

In [6]:
#read in path
path = f'../../../data/{FILE_NAME}'

In [7]:
def pre_process_daily_data(path, index_col, by_col, 
                           values, dayfirst=False):
    '''
    Daily data is stored in long format.  Read in 
    and pivot to wide format so that there is a single 
    colmumn for each regions time series.
    '''
    df = pd.read_csv(path, index_col=index_col, parse_dates=True, 
                     dayfirst=dayfirst)
    df.columns = map(str.lower, df.columns)
    df.index.rename(str(df.index.name).lower(), inplace=True)
    
    clean_table = pd.pivot_table(df, values=values.lower(), 
                                 index=[index_col.lower()],
                                 columns=[by_col.lower()], aggfunc=np.sum)
    
    clean_table.index.freq = 'D'
    
    return clean_table

In [8]:
clean = pre_process_daily_data(path, 'Actual_dt', 'ORA', 'Actual_Value', 
                               dayfirst=False)
clean.head()

ora,BNSSG,Cornwall,Devon,Dorset,Gloucestershire,OOA,Somerset,Trust,Wiltshire
actual_dt,,,,,,,,,
2013-12-30,415.0,220.0,502.0,336.0,129.0,NaN,183.0,2042.0,255.0
2013-12-31,420.0,236.0,468.0,302.0,128.0,NaN,180.0,1996.0,260.0
2014-01-01,549.0,341.0,566.0,392.0,157.0,NaN,213.0,2570.0,351.0
2014-01-02,450.0,218.0,499.0,301.0,115.0,NaN,167.0,2013.0,258.0
2014-01-03,419.0,229.0,503.0,304.0,135.0,NaN,195.0,2056.0,269.0


## Train Test Split

In [9]:
def ts_train_test_split(data, split_date):
    '''
    Split time series into training and test data
    
    Parameters:
    -------
    data - pd.DataFrame - time series data.  Index expected as datatimeindex
    split_date - the date on which to split the time series
    
    Returns:
    --------
    tuple (len=2) 
    0. pandas.DataFrame - training dataset
    1. pandas.DataFrame - test dataset
    '''
    train = data.loc[data.index < split_date]
    test = data.loc[data.index >= split_date]
    return train, test

In [10]:
train, test = ts_train_test_split(clean, split_date=TEST_SPLIT_DATE)

#exclude data after 2020 due to coronavirus.
test, discard = ts_train_test_split(test, split_date=DISCARD_DATE)

#split into train and val AFTER creating new years day.


In [11]:
train.shape

(1828, 9)

In [12]:
test.shape

(365, 9)

# Exogenous variables

Generate a new binary categorical feature representing new years day, calender dummies for month and day of week and lagged variables

In [13]:
lagged, calendar_dummies, new_year = featurize_time_series(train[REGION], 
                                                    max_lags=7, 
                                                    include_interactions=False)

In [14]:
#rename column and drop quarters and t from seasonal indexes
new_year.columns = ['new_year']
calendar_dummies = calendar_dummies[calendar_dummies.columns[:-4]]

In [15]:
new_year.head(3)

,new_year
actual_dt,
2013-12-30,0
2013-12-31,0
2014-01-01,1


In [16]:
calendar_dummies.head(3)

,m_2,m_3,m_4,m_5,m_6,m_7,m_8,m_9,m_10,m_11,m_12,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6
actual_dt,,,,,,,,,,,,,,,,,
2013-12-30,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0
2013-12-31,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0
2014-01-01,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0


In [17]:
#combined to single dataframe
processed = pd.concat([train[REGION], lagged, calendar_dummies, new_year], 
                      axis=1)
processed.head(3)

,actual,Trust_lag1,Trust_lag2,Trust_lag3,Trust_lag4,Trust_lag5,Trust_lag6,Trust_lag7,m_2,m_3,...,m_10,m_11,m_12,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,new_year
actual_dt,,,,,,,,,,,,,,,,,,,,,
2013-12-30,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,0,0,0,0,0,0,0
2013-12-31,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,1,1,0,0,0,0,0,0
2014-01-01,2570.0,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,0,0,...,0,0,0,0,1,0,0,0,0,1


# Train validation split

In [18]:
#train split into train and validation
train, val = ts_train_test_split(processed, split_date=VAL_SPLIT_DATE)

In [19]:
train.shape

(1279, 26)

In [20]:
val.shape

(549, 26)

In [21]:
#exog
train[train.columns[1:]].head(3)

,Trust_lag1,Trust_lag2,Trust_lag3,Trust_lag4,Trust_lag5,Trust_lag6,Trust_lag7,m_2,m_3,m_4,...,m_10,m_11,m_12,dow_1,dow_2,dow_3,dow_4,dow_5,dow_6,new_year
actual_dt,,,,,,,,,,,,,,,,,,,,,
2013-12-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,1,0,0,0,0,0,0,0
2013-12-31,2042.0,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,1,1,0,0,0,0,0,0
2014-01-01,1996.0,2042.0,NaN,NaN,NaN,NaN,NaN,0,0,0,...,0,0,0,0,1,0,0,0,0,1


# Auto ARIMA model selection

Uses Auto ARIMA function to select model by AIC


In [23]:
##remember that this contains lags - drop NAN rows.
MAX_LAG = 7

auto_results = auto_arima(train['actual'][MAX_LAG+1:], 
                          exogenous=train[train.columns[1:]][MAX_LAG+1:], 
                          seasonal=True, 
                          m=7, 
                          n_job=-1, 
                          suppress_warnings=False) 

In [24]:
auto_results.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                                      SARIMAX Results                                      
===========================================================================================
Dep. Variable:                                   y   No. Observations:                 1271
Model:             SARIMAX(1, 1, 2)x(0, 0, [1], 7)   Log Likelihood               -7188.052
Date:                             Fri, 05 Feb 2021   AIC                          14438.104
Time:                                     12:02:38   BIC                          14597.654
Sample:                                 01-07-2014   HQIC                         14498.037
                                      - 06-30-2017                                         
Covariance Type:                               opg                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
intercept      0.0327      0.067      0.485      0.628      -0.099       0.165
Trust_lag1    -0.4796      0.095     -5.023      0.000      -0.667      -0.292
Trust_lag2    -0.3577      0.063     -5.677      0.000      -0.481      -0.234
Trust_lag3    -0.2496      0.058     -4.332      0.000      -0.362      -0.137
Trust_lag4    -0.2071      0.052     -3.948      0.000      -0.310      -0.104
Trust_lag5    -0.1712      0.050     -3.440      0.001      -0.269      -0.074
Trust_lag6    -0.0836      0.043     -1.951      0.051      -0.167       0.000
Trust_lag7    -0.0290      0.035     -0.822      0.411      -0.098       0.040
m_2           15.5888     49.067      0.318      0.751     -80.581     111.759
m_3           26.2907     62.952      0.418      0.676     -97.093     149.675
m_4           -7.9284     69.971     -0.113      0.910    -145.068     129.211
m_5          -23.9935    129.327     -0.186      0.853    -277.469     229.483
m_6          -74.3069    127.478     -0.583      0.560    -324.160     175.546
m_7         -112.2881    127.594     -0.880      0.379    -362.367     137.791
m_8         -233.8662    115.205     -2.030      0.042    -459.663      -8.069
m_9         -295.6253    111.637     -2.648      0.008    -514.430     -76.820
m_10        -359.2385    106.122     -3.385      0.001    -567.234    -151.243
m_11        -235.6151    102.762     -2.293      0.022    -437.025     -34.205
m_12        -225.1194     53.035     -4.245      0.000    -329.066    -121.173
dow_1        -90.0485     10.355     -8.696      0.000    -110.344     -69.753
dow_2       -137.8469     18.981     -7.262      0.000    -175.050    -100.644
dow_3       -150.3362     23.955     -6.276      0.000    -197.287    -103.386
dow_4       -123.7627     25.082     -4.934      0.000    -172.923     -74.602
dow_5         20.1599     22.334      0.903      0.367     -23.614      63.934
dow_6         57.2658     10.168      5.632      0.000      37.337      77.195
new_year     241.2596     51.268      4.706      0.000     140.776     341.743
ar.L1          0.9099      0.043     21.307      0.000       0.826       0.994
ma.L1         -0.9365      0.100     -9.348      0.000      -1.133      -0.740
ma.L2         -0.0292      0.095     -0.308      0.758      -0.215       0.157
ma.S.L7       -0.0680      0.061     -1.111      0.267      -0.188       0.052
sigma2      4900.4491    182.822     26.804      0.000    4542.125    5258.774
===================================================================================
Ljung-Box (Q):                       36.44   Jarque-Bera (JB):                29.37
Prob(Q):                              0.63   Prob(JB):                         0.00
Heteroskedasticity (H):               1.08   Skew:                             0.20
Prob(H) (two-sided):                  0.44   Kurtosis:                         3.63
==================

## Test Out of Sample Prediction

In [25]:
print("pmdarima version: %s" % pm.__version__)
print(auto_results.order)

pmdarima version: 1.5.2
(1, 1, 2)


In [26]:
#override if desired to test different model.
order = auto_results.order
seasonal_order = auto_results.seasonal_order
#order = (1, 1, 2)
#seasonal_order = (0, 0, 1, 7)


In [27]:
model = pm.ARIMA(order=order, seasonal_order=seasonal_order)

In [32]:
model.fit(y=train['actual'][MAX_LAG+1:], 
          exogenous=train[train.columns[1:]][MAX_LAG+1:])

ARIMA(maxiter=50, method='lbfgs', order=(1, 1, 2), out_of_sample_size=0,
      scoring='mse', scoring_args=None, seasonal_order=(0, 0, 1, 7),
      start_params=None, suppress_warnings=False, trend=None,
      with_intercept=True)

In [33]:
#test prediction
model.predict(n_periods=5, 
              exogenous=val[val.columns[1:]].iloc[:5])

array([2252.64267482, 2147.63767524, 2008.40808841, 1954.66423768,
       1949.36818956])

# Wrapper classes for statsmodels ARIMA

Adapter/wrapper class to enable usage within standard cross validation used across all methods

In [42]:
class ARIMAWrapper:
    '''
    Facade for statsmodels ARIMA with exog
    '''
    def __init__(self, order, seasonal_order):
        self._order = order
        self._seasonal_order = seasonal_order

    def _get_resids(self):
        return self._fitted.resid

    def _get_preds(self):
        return self._fitted.fittedvalues

    def fit(self, train, exog=None):
        '''
        Fit the ARIMA model
        Here the intended use is to fit regression model with ARIMA errors
        
        Params:
        ------
        train: array-like
            training time series (univariate)
            
        exog: array-like, optional (default=None)
            exogenous regressors. Here the intended use is for seasonal indexes
            and special events such as holidays.
        
        '''
        self._model = ARIMA(endog=train,
                            exog=exog,
                            order=self._order, 
                            seasonal_order=self._seasonal_order,
                            enforce_stationarity=False)
        self._fitted = self._model.fit()
        self._t = len(train)
    
    def predict(self, horizon, exog=None, return_conf_int=False, alpha=0.2):
        '''
        forecast h steps ahead.
        
        Params:
        ------
        h: int
            h-step forecast
        
        exog: array-like, optional (default=None)
            Exogenous regressors.  Here typically seasonal indexes and
            special events.
        
        return_conf_int: bool, optional (default=False)
            return 1 - alpha PI
        
        alpha: float, optional (default=0.2)
            return 1 - alpha PI
                       
        Returns:
        -------
        np.array
            If return_conf_int = False returns preds only
            
        np.array, np.array
            If return_conf_int = True returns tuple of preds, pred_ints
        '''
        forecast = self._fitted.get_forecast(horizon, exog=exog)
        mean_forecast = forecast.summary_frame()['mean'].to_numpy()
        
        if return_conf_int:
            df = forecast.summary_frame(alpha=alpha)
            pi = df[['mean_ci_lower', 'mean_ci_upper']].to_numpy()
            return mean_forecast, pi
        
        else:
            return mean_forecast

    fittedvalues = property(_get_preds)
    resid = property(_get_resids)    
    

# Smoke test ARIMAWrapper prior to tscv

In [44]:
model = ARIMAWrapper(order=order, seasonal_order=seasonal_order)
model.fit(train=train['actual'][MAX_LAG+1:], 
          exog=train[train.columns[1:]][MAX_LAG+1:])

In [48]:
#test prediction
model.predict(horizon=5, 
              exog=val[val.columns[1:]].iloc[:5])

array([2251.9745008 , 2147.12520668, 2007.8307492 , 1953.99181565,
       1948.8978842 ])

In [50]:
#prediction intervals
#test prediction
forecast, intervals = model.predict(horizon=5, 
                                    exog=val[val.columns[1:]].iloc[:5],
                                    return_conf_int=True)

In [51]:
intervals

array([[2163.07744613, 2340.87155548],
       [2023.17457157, 2271.07584179],
       [1859.31249721, 2156.3490012 ],
       [1786.39583456, 2121.58779674],
       [1765.7458265 , 2132.04994191]])

## Cross Validation

`time_series_cv` implements rolling forecast origin cross validation for time series.  
It does not calculate forecast error, but instead returns the predictions, pred intervals and actuals in an array that can be passed to any forecast error function. (this is for efficiency and allows additional metrics to be calculated if needed).

In [52]:
def time_series_cv(model, train, val, horizons, alpha=0.2, step=1):
    '''
    Time series cross validation across multiple horizons for a single model.

    Incrementally adds additional training data to the model and tests
    across a provided list of forecast horizons. Note that function tests a
    model only against complete validation sets.  E.g. if horizon = 15 and 
    len(val) = 12 then no testing is done.  In the case of multiple horizons
    e.g. [7, 14, 28] then the function will use the maximum forecast horizon
    to calculate the number of iterations i.e if len(val) = 365 and step = 1
    then no. iterations = len(val) - max(horizon) = 365 - 28 = 337.
    
    Parameters:
    --------
    model - forecasting model

    train - np.array - vector of training data

    val - np.array - vector of validation data

    horizon - list of ints, forecast horizon e.g. [7, 14, 28] days

    step -- step taken in cross validation 
            e.g. 1 in next cross validation training data includes next point 
            from the validation set.
            e.g. 7 in the next cross validation training data includes next 7 points
            (default=1)
            
    Returns:
    -------
    np.array - vector of forecast errors from the CVs.
    '''
    cv_preds = [] #mean forecast
    cv_actuals = [] # actuals 
    cv_pis = [] #prediction intervals
    split = 0

    print('split => ', end="")
    for i in range(0, len(val) - max(horizons) + 1, step):
        split += 1
        print(f'{split}, ', end="")
        
        #create new training y value and exogenous variables
        y_train = np.concatenate([train.iloc[:,0], val.iloc[:i,0]], axis=0)   
        X_train = np.concatenate([train.iloc[:,1:], 
                                  val.iloc[:i,1:]], axis=0)
                
        model.fit(y_train, exog=X_train)
        
        #create validation X values.
        horizon=len(val[i:i+max(horizons)])
        X_val = val.iloc[i:i+horizon,1:]
        
        #predict the maximum horizon 
        preds, pis = model.predict(horizon=horizon, 
                                   exog=X_val,
                                   return_conf_int=True,
                                   alpha=alpha)
        
        cv_h_preds = []
        cv_test = []
        cv_h_pis = []
        
        for h in horizons:
            #store the h-step prediction
            cv_h_preds.append(preds[:h])
            #store the h-step actual value
            cv_test.append(val.iloc[i:i+h, 0])    
            cv_h_pis.append(pis[:h])
                     
        cv_preds.append(cv_h_preds)
        cv_actuals.append(cv_test)
        cv_pis.append(cv_h_pis)
        
    print('done.\n')        
    return cv_preds, cv_actuals, cv_pis

## Custom functions for calculating CV scores for point predictions and coverage.

These functions have been written to work with the output of `time_series_cv`

In [53]:
def split_cv_error(cv_preds, cv_test, error_func):
    '''
    Forecast error in the current split
    
    Params:
    -----
    cv_preds, np.array
        Split predictions
        
    
    cv_test: np.array
        acutal ground truth observations
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
        np.ndarray
            cross validation errors for split
    '''
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = error_func(cv_test[split], cv_preds[split])
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv(cv_preds, cv_test, error_func):
    '''
    Forecast errors by forecast horizon
    
    Params:
    ------
    cv_preds: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    cv_test: np.ndarray
        Array of arrays.  Each array is of size h representing
        the forecast horizon specified.
        
    error_func: object
        function with signature (y_true, y_preds)
        
    Returns:
    -------
    np.ndarray
        
    '''
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error(cv_preds[h], cv_test[h], error_func)
        horizon_errors.append(split_errors)

    return np.array(horizon_errors)

def split_coverage(cv_test, cv_intervals):
    n_splits = len(cv_test)
    cv_errors = []
        
    for split in range(n_splits):
        val = np.asarray(cv_test[split])
        lower = cv_intervals[split].T[0]
        upper = cv_intervals[split].T[1]
        
        coverage = len(np.where((val > lower) & (val < upper))[0])
        coverage = coverage / len(val)
        
        cv_errors.append(coverage)
        
    return np.array(cv_errors)
    
    
def prediction_int_coverage_cv(cv_test, cv_intervals):
    cv_test = np.array(cv_test)
    cv_intervals = np.array(cv_intervals)
    n_horizons = len(cv_test)    
    
    horizon_coverage = []
    for h in range(n_horizons):
        split_coverages = split_coverage(cv_test[h], cv_intervals[h])
        horizon_coverage.append(split_coverages)

    return np.array(horizon_coverage)  

In [54]:
def split_cv_error_scaled(cv_preds, cv_test, y_train):
    n_splits = len(cv_preds)
    cv_errors = []
    
    for split in range(n_splits):
        pred_error = mean_absolute_scaled_error(cv_test[split], cv_preds[split], 
                                                y_train, period=7)
        
        cv_errors.append(pred_error)
        
    return np.array(cv_errors)

def forecast_errors_cv_scaled(cv_preds, cv_test, y_train):
    cv_test = np.array(cv_test)
    cv_preds = np.array(cv_preds)
    n_horizons = len(cv_test)    
    
    horizon_errors = []
    for h in range(n_horizons):
        split_errors = split_cv_error_scaled(cv_preds[h], cv_test[h], y_train)
        horizon_errors.append(split_errors)
        
    return np.array(horizon_errors)

# Run cross validation

This is run twices once each for 80 and 95% prediction intervals.

In [55]:
#reminder of ARIMA order
print(order)
print(seasonal_order)

(1, 1, 2)
(0, 0, 1, 7)


### Create model and run

In [63]:
MAX_LAG = 7
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
STEP = 7

#create model
model = ARIMAWrapper(order=order, seasonal_order=seasonal_order)

#run tscv
results  = time_series_cv(model=model,
                          train=train[MAX_LAG+1:], 
                          val=val, 
                          horizons=horizons,
                          step=STEP,
                          alpha=0.2)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



## symmetric MAPE results

In [64]:
cv_preds, cv_test, cv_intervals = results

In [65]:
#CV point predictions smape
cv_errors = forecast_errors_cv(cv_preds, cv_test, 
                               symmetric_mean_absolute_percentage_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,4.570023,6.098915,7.039471,7.798917,8.509989,9.182582,9.776427,10.315478,10.826109,11.247215,11.651120,12.067720,11.498736
std,2.490329,3.127826,3.455725,3.649714,3.986365,4.208956,4.204118,3.991195,3.752904,3.432351,3.062639,2.886313,3.439675
min,1.356336,2.398354,2.976610,3.375812,3.679565,3.724181,4.375734,4.233545,4.446652,4.813858,5.895136,7.201263,7.132087
25%,2.887900,4.072286,4.585279,5.370347,5.258651,5.428414,5.824347,6.589455,7.762340,8.729194,9.657538,10.418858,8.541598
50%,4.050446,5.007924,6.129879,6.686204,7.906887,8.540617,10.755881,11.648903,11.564629,11.424611,11.517712,11.498327,11.834270
75%,5.328956,7.294972,7.401325,9.542920,11.472288,11.586925,12.635047,13.154315,13.372610,13.233846,13.665600,14.121907,14.065354
max,12.100541,14.186901,17.513817,18.226541,19.344479,19.949988,20.149281,19.610335,20.240158,19.778800,19.362634,20.001932,20.373477


In [66]:
#output sMAPE results to file
metric = 'smape'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima2_smape.csv


## RMSE results

In [67]:
#CV point predictions rmse - no interactions
cv_errors = forecast_errors_cv(cv_preds, cv_test, root_mean_squared_error)
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,114.161851,156.877098,183.165590,202.788646,220.221891,236.548098,251.115588,264.484678,277.966409,289.233335,300.011061,310.980641,314.046618
std,56.821253,83.872862,93.467036,96.615123,103.314578,106.988571,106.160576,100.536958,93.916279,83.738422,71.422501,63.809712,78.752861
min,45.557536,63.720984,78.402010,85.621644,100.893490,100.600888,116.146711,117.574487,120.450988,127.865624,161.896566,201.113572,219.559619
25%,74.801314,109.462791,127.500798,134.933786,137.106879,144.779506,155.671096,174.320796,196.748279,231.187210,272.625002,288.848030,243.534113
50%,100.204710,125.344073,156.363017,170.777837,197.445298,219.288645,274.981025,294.464954,300.587013,304.776704,305.860612,314.262430,318.095587
75%,123.210368,180.288023,183.558756,255.619365,303.394186,314.934227,323.025405,327.613594,334.099129,336.855104,336.231506,337.611058,368.828966
max,298.866758,418.094358,498.505925,506.480516,528.383391,539.948774,541.153250,526.277701,540.945002,529.908938,519.042460,535.658511,541.511133


In [68]:
#output rmse
metric = 'rmse'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima2_rmse.csv


## MASE results

In [70]:
#mase
cv_errors = forecast_errors_cv_scaled(cv_preds, cv_test, train['actual'])
df = pd.DataFrame(cv_errors)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,1.207769,1.624940,1.880557,2.083165,2.269348,2.444496,2.598092,2.735171,2.865207,2.970976,3.071364,3.175808,3.052776
std,0.674839,0.895577,1.021226,1.086839,1.174836,1.229185,1.224569,1.160423,1.098175,1.005111,0.897011,0.844507,0.945453
min,0.394687,0.608625,0.753756,0.846671,0.956472,0.963416,1.139646,1.087285,1.135450,1.237500,1.499993,1.816609,1.910415
25%,0.761537,1.055836,1.235542,1.371702,1.358352,1.372798,1.546121,1.748307,1.970431,2.268311,2.563567,2.739923,2.369221
50%,1.033168,1.389323,1.555127,1.763345,2.123809,2.185585,2.643202,3.076600,3.135188,3.124357,3.174802,3.029090,3.054106
75%,1.373741,1.866659,1.952958,2.501649,3.138311,3.228887,3.376724,3.307773,3.532221,3.449581,3.494335,3.605765,3.584492
max,3.385775,4.301980,5.352621,5.584451,5.919867,6.112454,6.170641,6.000814,6.189045,6.052496,5.925983,6.119555,6.117247


In [71]:
metric = 'mase'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima2_mase.csv


## 80% Coverage

In [72]:
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.783069,0.764550,0.744268,0.714286,0.686772,0.660494,0.641723,0.628968,0.620223,0.610053,0.596441,0.582892,0.788737
std,0.292154,0.279335,0.267799,0.268726,0.278196,0.283153,0.286069,0.280137,0.256434,0.234330,0.211564,0.198203,0.115984
min,0.000000,0.071429,0.095238,0.071429,0.057143,0.047619,0.040816,0.053571,0.047619,0.085714,0.116883,0.107143,0.438356
25%,0.714286,0.678571,0.642857,0.589286,0.542857,0.500000,0.459184,0.410714,0.444444,0.450000,0.415584,0.410714,0.706849
50%,0.857143,0.857143,0.857143,0.821429,0.771429,0.738095,0.653061,0.625000,0.650794,0.657143,0.610390,0.630952,0.780822
75%,1.000000,0.928571,0.904762,0.928571,0.914286,0.916667,0.908163,0.875000,0.841270,0.807143,0.779221,0.738095,0.897260
max,1.000000,1.000000,1.000000,1.000000,1.000000,0.976190,0.959184,0.964286,0.968254,0.957143,0.896104,0.833333,0.934247


In [73]:
#write 80% coverage to file
metric = 'coverage_80'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima2_coverage_80.csv


# Repeat for 95% PIs.

In [74]:
#95% pis
MAX_LAG = 7
horizons = [7, 14, 21, 28, 35, 42, 49, 56, 63, 70, 77, 84, 365]
STEP = 7

#create model
model = ARIMAWrapper(order=order, seasonal_order=seasonal_order)

#run tscv
results  = time_series_cv(model=model,
                          train=train[MAX_LAG+1:], 
                          val=val, 
                          horizons=horizons,
                          step=STEP,
                          alpha=0.05)

split => 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, done.



## 95% coverage results

In [75]:
cv_preds, cv_test, cv_intervals = results

In [76]:
cv_coverage = prediction_int_coverage_cv(cv_test, cv_intervals)
df = pd.DataFrame(cv_coverage)
df.columns = horizons
df.describe()

,7,14,21,28,35,42,49,56,63,70,77,84,365
count,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000,27.000000
mean,0.931217,0.910053,0.897707,0.892857,0.885714,0.877425,0.873772,0.875000,0.873016,0.872487,0.874940,0.873898,0.927144
std,0.155336,0.180041,0.190340,0.195866,0.191990,0.190900,0.186618,0.161627,0.160370,0.145088,0.130823,0.131213,0.058268
min,0.285714,0.357143,0.238095,0.178571,0.142857,0.119048,0.102041,0.214286,0.190476,0.257143,0.324675,0.297619,0.720548
25%,0.928571,0.928571,0.928571,0.875000,0.814286,0.797619,0.816327,0.830357,0.809524,0.828571,0.844156,0.857143,0.891781
50%,1.000000,1.000000,1.000000,1.000000,0.971429,0.952381,0.938776,0.892857,0.888889,0.885714,0.896104,0.892857,0.953425
75%,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.992063,0.964286,0.941558,0.922619,0.967123
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,0.978082


In [77]:
#write 95% coverage to file
metric = 'coverage_95'
print(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')
df.to_csv(f'{TOP_LEVEL}/{STAGE}/{REGION}-{METHOD}_{metric}.csv')

../../../results/model_selection/stage1/Trust-reg-arima2_coverage_95.csv


# End